In [ ]:

import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
root_dir = "/kaggle/input/corn-leaf-infection-dataset/Corn Disease detection"
os.listdir(root_dir)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
# hyper parameters
batch_size = 32
img_height, img_width=256,256
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    root_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    root_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

In [ ]:
validation_generator[0][1][1]

In [ ]:
def show(img, title=None, fig_size=(5, 5)):
    fig = plt.figure(figsize=fig_size)
    if title is not None:
        plt.title(title)
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
import cv2
plt.style.use('seaborn-whitegrid')
for i in range(3):
    labels = ["Normal", "Infected"]
    img = validation_generator[0][0][i]
    lbl = labels[np.argmax(validation_generator[0][1][i])]
    
    img = cv2.resize(img, (img_height, img_width))
    img = img.reshape(img_height, img_width, 3)
    show (img, title=lbl)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (img_height, img_width, 3), padding="same", activation = 'relu', data_format = 'channels_last'))
model.add(Conv2D(32, (3, 3), activation='relu', padding="same"))
model.add(MaxPool2D(pool_size=(3, 3)))
# model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(MaxPool2D(pool_size=(3, 3)))

model.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
model.add(MaxPool2D(pool_size=(3, 3)))

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

In [ ]:
lr=1e-03
epochs=15

opt = Adam(lr=lr, decay=lr / epochs)

# compile it !
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

# train it
history=model.fit_generator(
train_generator,
steps_per_epoch=len(train_generator),
validation_data=validation_generator,
validation_steps=len(validation_generator),
epochs=epochs,
verbose=1,
workers=32)

In [ ]:
loss,accuracy=model.evaluate(x_test,y_test)